## Simple Allo -> DSLX flow (stateless dataflow)

In [1]:
%load_ext autoreload
%autoreload 2
import allo
from allo.ir.types import int32, uint32

In [2]:
def add(a: uint32, b: uint32) -> uint32:
  return a + b

s = allo.customize(add)
code = s.build(target='xls')
# print(code)

In [3]:
# Validate add proc
for lhs, rhs, expected in [(0, 0, 0), (1, 2, 3), (123, 456, 579), (2**16, 2**16, 2**17)]:
  code.test(lhs, rhs, expected)


[ RUN UNITTEST  ] add_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] add_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] add_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] add_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [4]:
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/add.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [5]:
code.to_ir()

package add

file_number 0 "abax/add.x"

chan add__in0(bits[32], id=0, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan add__in1(bits[32], id=1, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan add__out0(bits[32], id=2, kind=streaming, ops=send_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)

top proc __add__add_0_next(__state: (), init={()}) {
  after_all.4: token = after_all(id=4)
  literal.3: bits[1] = literal(value=1, id=3)
  after_all.9: token = after_all(id=9)
  receive.5: (token, bits[32]) = receive(after_all.4, predicate=literal.3, channel=add__in0, id=5)
  receive.10: (token, bits[32]) = receive(after_all.9, predicate=literal.3, channel=add__in1, id=10)
  tmp0: bits[32] = tuple_index(receive.5, index=1, id=8, pos=[(0,10,15)])
  tmp1: bits[32] = tuple_index(receive.10, index=1, id=13, pos=[(0,11,15)])
  tmp2: bits[33] = zero_ext(tmp0, new_bit_count=3

In [6]:
code.opt()

package add

file_number 0 "abax/add.x"

chan add__in0(bits[32], id=0, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan add__in1(bits[32], id=1, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan add__out0(bits[32], id=2, kind=streaming, ops=send_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)

top proc __add__add_0_next() {
  after_all.4: token = after_all(id=4)
  receive.37: (token, bits[32]) = receive(after_all.4, channel=add__in0, id=37)
  receive.38: (token, bits[32]) = receive(after_all.4, channel=add__in1, id=38)
  tok0: token = tuple_index(receive.37, index=0, id=7, pos=[(0,10,9)])
  tok1: token = tuple_index(receive.38, index=0, id=12, pos=[(0,11,9)])
  tmp0: bits[32] = tuple_index(receive.37, index=1, id=8, pos=[(0,10,15)])
  tmp1: bits[32] = tuple_index(receive.38, index=1, id=13, pos=[(0,11,15)])
  tok: token = after_all(tok0, tok1, id=18)
  tmp4__

In [7]:
code.to_vlog()

module __add__add_0_next(
  input wire clk,
  input wire rst,
  input wire [31:0] add__in0,
  input wire add__in0_vld,
  input wire [31:0] add__in1,
  input wire add__in1_vld,
  input wire add__out0_rdy,
  output wire add__in0_rdy,
  output wire add__in1_rdy,
  output wire [31:0] add__out0,
  output wire add__out0_vld
);
  reg [31:0] __add__in0_reg;
  reg __add__in0_valid_reg;
  reg [31:0] __add__in1_reg;
  reg __add__in1_valid_reg;
  reg [31:0] __add__out0_reg;
  reg __add__out0_valid_reg;
  wire add__out0_valid_inv;
  wire p0_all_active_inputs_valid;
  wire add__out0_valid_load_en;
  wire add__out0_load_en;
  wire p0_stage_done;
  wire add__in0_valid_inv;
  wire add__in1_valid_inv;
  wire add__in0_valid_load_en;
  wire add__in1_valid_load_en;
  wire add__in0_load_en;
  wire add__in1_load_en;
  wire [31:0] tmp4__1;
  assign add__out0_valid_inv = ~__add__out0_valid_reg;
  assign p0_all_active_inputs_valid = __add__in0_valid_reg & __add__in1_valid_reg;
  assign add__out0_valid_load_en =

In [8]:
code.flow()

## Some other examples

In [65]:
# supports both unsigned and signed integers
def mac(a: int32, b: int32, c: int32) -> int32:
  return (a * b) + c

s = allo.customize(mac)
print(s.module)
code = s.build(target='xls')
# print(code)
code.flow()

module {
  func.func @mac(%arg0: i32, %arg1: i32, %arg2: i32) -> i32 attributes {itypes = "sss", otypes = "s"} {
    %0 = arith.extsi %arg0 : i32 to i64
    %1 = arith.extsi %arg1 : i32 to i64
    %2 = arith.muli %0, %1 : i64
    %3 = arith.extsi %2 : i64 to i65
    %4 = arith.extsi %arg2 : i32 to i65
    %5 = arith.addi %3, %4 : i65
    %6 = arith.trunci %5 : i65 to i32
    return %6 : i32
  }
}



In [10]:
# Validate mac proc
for a, b, c, expected in [(2, 3, 4, 10), (5, -1, 7, 2), (-4, 5, 6, -14)]:
  code.test(a, b, c, expected)


[ RUN UNITTEST  ] mac_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] mac_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] mac_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [11]:
# supports multiple outputs
def wsa(a: int32, b: int32) -> (int32, int32, int32):
  return a | b, a & b, a ^ b

s = allo.customize(wsa)
code = s.build(target='xls')
# print(code)
code.flow()

In [12]:
# Validate wsa proc
for a, b in [(0, 0), (5, 3), (-1, 7)]:
  code.test(a, b, a | b, a & b, a ^ b)


[ RUN UNITTEST  ] wsa_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] wsa_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] wsa_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [13]:
# supports (basic) conditional statements
def max(a: int32, b: int32) -> int32:
  return a if (a > b) else b

s = allo.customize(max)
code = s.build(target='xls')
# print(code)
# code.flow()

In [14]:
# Validate max proc
for lhs, rhs, expected in [(-3, -7, -3), (10, 4, 10), (5, 9, 9)]:
  code.test(lhs, rhs, expected)


[ RUN UNITTEST  ] max_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] max_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] max_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [15]:
# supports (basic) conditional statements
def incr(a: int32) -> int32:
  return a + 1

s = allo.customize(incr)
code = s.build(target='xls')
# print(code)
# code.flow()

In [16]:
# Validate incr proc
for value, expected in [(-1, 0), (0, 1), (41, 42), (2**10, 2**10 + 1)]:
  code.test(value, expected)


[ RUN UNITTEST  ] incr_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] incr_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] incr_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] incr_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [17]:
# Generate XLS/DSLX code for fact function
def fact(a: int32) -> int32:
  acc: int32 = 1
  for i in range(a):
    acc *= (i + 1)
  return acc

s = allo.customize(fact)
# print(s.module)
code = s.build(target='xls')
# print(code)
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/fact.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [18]:
# Validate fact proc
for value, expected in [(0, 1), (1, 1), (5, 120), (7, 5040), (10, 3628800)]:
  code.test(value, expected)


[ RUN UNITTEST  ] fact_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fact_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fact_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fact_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fact_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [19]:
# Generate XLS/DSLX code for fibonacci function
# This tests multiple accumulators (prev, curr)
def fib(n: int32) -> int32:
  prev: int32 = 0
  curr: int32 = 1
  for i in range(n):
    next_val: int32 = prev + curr
    prev = curr
    curr = next_val
  return curr

s = allo.customize(fib)
# print(s.module)
code = s.build(target='xls')
# print(code)
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/fib.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [20]:
# Validate fib proc
for n, expected in [(0, 1), (1, 1), (5, 8), (10, 89)]:
  code.test(n, expected)


[ RUN UNITTEST  ] fib_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fib_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fib_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] fib_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [21]:
# WHILE loop example: Count steps until n becomes 1 (Collatz-like)
# Simplified: divide by 2 if even, subtract 1 if odd, count steps
def count_steps(n: int32) -> int32:
  steps: int32 = 0
  val: int32 = n
  while val > 1:
    if val % 2 == 0:
      val = val // 2
    else:
      val = val - 1
    steps = steps + 1
  return steps

s = allo.customize(count_steps)
# print("=== MLIR for count_steps (WHILE loop) ===")
# print(s.module)
# print("\n=== Building DSLX ===")
code = s.build(target='xls')
# print(code)
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/count_steps.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [22]:
# Validate count_steps proc
for value, expected in [(1, 0), (2, 1), (4, 2), (7, 4), (16, 4)]:
  code.test(value, expected)


[ RUN UNITTEST  ] count_steps_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] count_steps_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] count_steps_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] count_steps_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] count_steps_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [23]:
# Nested FOR loop example: Sum of products (simulates 2D iteration)
# Computes sum of i*j for i in range(n), j in range(m)
def nested_sum(n: int32, m: int32) -> int32:
  total: int32 = 0
  for i in range(n):
    for j in range(m):
      total = total + (i * j)
  return total

s = allo.customize(nested_sum)
# print("=== MLIR for nested_sum (nested FOR loops) ===")
# print(s.module)
# print("\n=== Building DSLX ===")
code = s.build(target='xls')
# print(code)
code.interpret()


abax/nested_sum.x:14:9-14:13
0012:     let (tok0, tmp0) = recv_if(join(), in0, !busy, ub0);
0013:     let (tok1, tmp1) = recv_if(join(), in1, !busy, ub1);
0014:     let tmp2 = (tmp1 as s32);
~~~~~~~~~~~~~~^--^ Definition of `tmp2` (type `sN[32]`) is not used in function `nested_sum.next`
0015:     let tmp3 = (index0 * index1);
0016:     let tmp4 = (acc0 as s34);



CompletedProcess(args=['interpreter_main', 'abax/nested_sum.x'], returncode=1, stdout='', stderr='abax/nested_sum.x:14:9-14:13\n0012:     let (tok0, tmp0) = recv_if(join(), in0, !busy, ub0);\n0013:     let (tok1, tmp1) = recv_if(join(), in1, !busy, ub1);\n0014:     let tmp2 = (tmp1 as s32);\n~~~~~~~~~~~~~~^--^ Definition of `tmp2` (type `sN[32]`) is not used in function `nested_sum.next`\n0015:     let tmp3 = (index0 * index1);\n0016:     let tmp4 = (acc0 as s34);\n')

In [24]:
# Validate nested_sum proc
for n, m, expected in [(2, 3, 3), (3, 3, 9), (1, 5, 0), (4, 4, 36)]:
  code.test(n, m, expected)


abax/nested_sum.x:14:9-14:13
0012:     let (tok0, tmp0) = recv_if(join(), in0, !busy, ub0);
0013:     let (tok1, tmp1) = recv_if(join(), in1, !busy, ub1);
0014:     let tmp2 = (tmp1 as s32);
~~~~~~~~~~~~~~^--^ Definition of `tmp2` (type `sN[32]`) is not used in function `nested_sum.next`
0015:     let tmp3 = (index0 * index1);
0016:     let tmp4 = (acc0 as s34);


abax/nested_sum.x:14:9-14:13
0012:     let (tok0, tmp0) = recv_if(join(), in0, !busy, ub0);
0013:     let (tok1, tmp1) = recv_if(join(), in1, !busy, ub1);
0014:     let tmp2 = (tmp1 as s32);
~~~~~~~~~~~~~~^--^ Definition of `tmp2` (type `sN[32]`) is not used in function `nested_sum.next`
0015:     let tmp3 = (index0 * index1);
0016:     let tmp4 = (acc0 as s34);


abax/nested_sum.x:14:9-14:13
0012:     let (tok0, tmp0) = recv_if(join(), in0, !busy, ub0);
0013:     let (tok1, tmp1) = recv_if(join(), in1, !busy, ub1);
0014:     let tmp2 = (tmp1 as s32);
~~~~~~~~~~~~~~^--^ Definition of `tmp2` (type `sN[32]`) is not used in fun

In [25]:
# GCD using WHILE loop (Euclidean algorithm)
def gcd(a: int32, b: int32) -> int32:
  x: int32 = a
  y: int32 = b
  while y > 0:
    temp: int32 = y
    y = x % y
    x = temp
  return x

s = allo.customize(gcd)
# print("=== MLIR for gcd (WHILE loop with two state vars) ===")
# print(s.module)
# print("\n=== Building DSLX ===")
code = s.build(target='xls')
# print(code)
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/gcd.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [26]:
# Validate gcd proc using XLSModule.test()
for a, b, expected in [(12, 8, 4), (48, 18, 6), (17, 13, 1), (100, 25, 25), (7, 7, 7)]:
  # print(f"gcd({a}, {b}) -> expecting {expected}")
  code.test(a, b, expected)


[ RUN UNITTEST  ] gcd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] gcd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] gcd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] gcd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] gcd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [27]:
# Nested loop with outer-loop preamble logic
# Each row computes a base offset before entering the inner loop

def nested_with_offset(n: int32, m: int32) -> int32:
  total: int32 = 0
  for i in range(n):
    row_base: int32 = i * 7 + 3  # outer-loop logic before inner traversal
    for j in range(m):
      total = total + row_base + j
  return total

s = allo.customize(nested_with_offset)
# print(s.module)
code = s.build(target='xls')
# print(code)
code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/nested_with_offset.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [28]:
# Validate nested_with_offset proc
for n, m, expected in [
  (0, 5, 0),
  (1, 1, 3),
  (2, 2, 28),
  (3, 4, 138)
]:
  code.test(n, m, expected)


[ RUN UNITTEST  ] nested_with_offset_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] nested_with_offset_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] nested_with_offset_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.


[ RUN UNITTEST  ] nested_with_offset_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



In [48]:
# vector-vector add

def vvadd(a: int32[32], b: int32[32]) -> int32[32]:
  c: int32[32] = 0
  for i in range(32):
    c[i] = a[i] + b[i]
  return c

s = allo.customize(vvadd)
# print(s.module)
code = s.build(target='xls')
# print(code)
code.interpret()
code.to_ir(False)
code.opt()


[===============] 0 test(s) ran; 0 failed; 0 skipped.

package vvadd

file_number 0 "abax/vvadd.x"

chan vvadd__mem0__read_req((bits[5]), id=0, kind=streaming, ops=send_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__mem0__read_resp((bits[32]), id=1, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__mem1__read_req((bits[5]), id=2, kind=streaming, ops=send_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__mem1__read_resp((bits[32]), id=3, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__mem2__write_req((bits[5], bits[32]), id=4, kind=streaming, ops=send_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__mem2__write_resp((), id=5, kind=streaming, ops=receive_only, flow_control=ready_valid, strictness=proven_mutually_exclusive)
chan vvadd__go(bits[1], id=6, kind=streaming, 

In [46]:
import numpy as np

# Validate vvadd memory-based proc
vec_len = 32
vec_a = np.arange(vec_len, dtype=np.int32)
vec_b = np.arange(vec_len, dtype=np.int32) * 2
expected = vec_a + vec_b
code.test(vec_a, vec_b, expected)


[ RUN UNITTEST  ] vvadd_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/vvadd.x'], returncode=0, stdout='', stderr='[ RUN UNITTEST  ] vvadd_test\n[            OK ]\n[===============] 1 test(s) ran; 0 failed; 0 skipped.\n')

In [63]:
def mv[N](A: int32[N, N], x: int32[N]) -> int32[N]:
  C: int32[N] = 0
  for i in range(N):
    acc: int32 = 0
    for j in range(N):
      acc += A[i, j] * x[j]
    C[i] = acc
  return C

mv_sched = allo.customize(mv, instantiate=[4])
mv_code = mv_sched.build(target='xls')
mv_code.interpret()


[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/mv.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [64]:
# Validate mv matvec
mat4 = (np.arange(16, dtype=np.int32).reshape(4, 4) - 3)
vec4 = np.array([1, -2, 3, -4], dtype=np.int32)
expected_vec = mat4 @ vec4
mv_code.test(mat4, vec4, expected_vec)



[ RUN UNITTEST  ] mv_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/mv.x'], returncode=0, stdout='', stderr='[ RUN UNITTEST  ] mv_test\n[            OK ]\n[===============] 1 test(s) ran; 0 failed; 0 skipped.\n')

In [53]:
def mm[N](A: int32[N, N], B: int32[N, N]) -> int32[N, N]:
  C: int32[N, N] = 0
  for i, j, k in allo.grid(N, N, N):
    C[i, j] += A[i, k] * B[k, j]
  return C

mm_sched = allo.customize(mm, instantiate=[4])
mm_code = mm_sched.build(target='xls')
mm_code.interpret()



[===============] 0 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/mm.x'], returncode=0, stdout='', stderr='[===============] 0 test(s) ran; 0 failed; 0 skipped.\n')

In [54]:
# Validate mm matrix multiply
mat_a = (np.arange(16, dtype=np.int32).reshape(4, 4) - 1)
mat_b = (np.arange(16, dtype=np.int32).reshape(4, 4) + 2)
expected_mat = mat_a @ mat_b
mm_code.test(mat_a, mat_b, expected_mat)


[ RUN UNITTEST  ] mm_test
[            OK ]
[===============] 1 test(s) ran; 0 failed; 0 skipped.



CompletedProcess(args=['interpreter_main', 'abax/mm.x'], returncode=0, stdout='', stderr='[ RUN UNITTEST  ] mm_test\n[            OK ]\n[===============] 1 test(s) ran; 0 failed; 0 skipped.\n')